In [1]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [6]:
load_dotenv(override=True)
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
openrouter_api_key

In [7]:
tell_a_joke = [
    {"role": "user", "content": "Tell a joke for a student on the journey to becoming an expert in LLM Engineering"},
]

In [8]:
openai = OpenAI()
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

# response = openrouter.chat.completions.create(model="x-ai/grok-4.1-fast:free", messages=tell_a_joke)
# display(Markdown(response.choices[0].message.content))

In [9]:
response = ollama.chat.completions.create(model="gpt-oss:latest", messages=tell_a_joke)
display(Markdown(response.choices[0].message.content))

Why did the LLM student bring a flashlight to the lab?

Because when the dataset shrouded them in code‑darkness, they had to *illuminate* the attention weights to see the path forward!

In [11]:
system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = ollama.chat.completions.create(model="gpt-oss:latest", messages=messages)
    return response.choices[0].message.content

In [12]:
from scraper import fetch_website_contents
import gradio as gr

In [13]:

# Again this is typical Experimental mindset - I'm changing the global variable we used above:

system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [14]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python guide in the guides folder

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat.completions.create(
        model='gpt-oss:latest',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [15]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    result = stream_gpt(prompt)
    yield from result

In [ ]:
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL including http:// or https://")
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_brochure,
    title="Brochure Generator", 
    inputs=[name_input, url_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Edward Donner", "https://edwarddonner.com", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
